In [33]:
%pip install faker



   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   -------------------------------------- - 1.8/1.9 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 13.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Data Preprocess


## Import Libraries


In [1]:

# # Data Preprocess for Category Classificatin
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from collections import Counter
import re
import random
from faker import Faker



## Initialize Dataset


In [3]:
file_path = '../dataset/dataset.csv'


In [4]:

# Load dataset from CSV file
df = pd.read_csv(file_path)

df = df.drop(columns=['flags'])

# Display the first few rows of the dataframe
df.head()


,instruction,category,intent,response
0,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [48]:
# Check the total length of the dataframe
total_length = len(df)
print(f"Total length of the dataframe: {total_length}")

# Check for any missing or empty values in each column
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

# Check for any missing or empty values in each row
missing_values_rows = df.isnull().sum(axis=1)
print("Missing values in each row:")
print(missing_values_rows)


Total length of the dataframe: 26872
Missing values in each column:
instruction    0
category       0
intent         0
response       0
dtype: int64
Missing values in each row:
0        0
1        0
2        0
3        0
4        0
        ..
26867    0
26868    0
26869    0
26870    0
26871    0
Length: 26872, dtype: int64


## Checking of Dataset


In [49]:
unique_intents = df['intent'].unique()
unique_categories = df['category'].unique()

print("Unique intents:")
print(unique_intents)
print("Total unique intents:", len(unique_intents))

print("\nUnique categories:")
print(unique_categories)
print("Total unique categories:", len(unique_categories))


Unique intents:
['cancel_order' 'change_order' 'change_shipping_address'
 'check_cancellation_fee' 'check_invoice' 'check_payment_methods'
 'check_refund_policy' 'complaint' 'contact_customer_service'
 'contact_human_agent' 'create_account' 'delete_account'
 'delivery_options' 'delivery_period' 'edit_account' 'get_invoice'
 'get_refund' 'newsletter_subscription' 'payment_issue' 'place_order'
 'recover_password' 'registration_problems' 'review'
 'set_up_shipping_address' 'switch_account' 'track_order' 'track_refund']
Total unique intents: 27

Unique categories:
['ORDER' 'SHIPPING' 'CANCEL' 'INVOICE' 'PAYMENT' 'REFUND' 'FEEDBACK'
 'CONTACT' 'ACCOUNT' 'DELIVERY' 'SUBSCRIPTION']
Total unique categories: 11


## Category and Intent


Intent


In [50]:
# Get the unique intents
unique_intents_list = df['intent'].unique()

# Print the unique intents
print("Unique intents:")
print(unique_intents_list)

# Get the distribution of intents
intent_distribution = df['intent'].value_counts()

# Print the distribution of intents
print("\nIntent distribution:")
print(intent_distribution)

print("Total unique intents:", len(unique_intents_list))


Unique intents:
['cancel_order' 'change_order' 'change_shipping_address'
 'check_cancellation_fee' 'check_invoice' 'check_payment_methods'
 'check_refund_policy' 'complaint' 'contact_customer_service'
 'contact_human_agent' 'create_account' 'delete_account'
 'delivery_options' 'delivery_period' 'edit_account' 'get_invoice'
 'get_refund' 'newsletter_subscription' 'payment_issue' 'place_order'
 'recover_password' 'registration_problems' 'review'
 'set_up_shipping_address' 'switch_account' 'track_order' 'track_refund']

Intent distribution:
intent
edit_account                1000
switch_account              1000
check_invoice               1000
complaint                   1000
contact_customer_service    1000
delivery_period              999
registration_problems        999
check_payment_methods        999
contact_human_agent          999
payment_issue                999
newsletter_subscription      999
get_invoice                  999
place_order                  998
cancel_order        

Category


In [51]:
# Get the unique categories
unique_categories_list = df['category'].unique()

# Print the unique categories
print("Unique categories:")
print(unique_categories_list)

# Get the distribution of categories
category_distribution = df['category'].value_counts()

# Print the distribution of categories
print("\nCategory distribution:")
print(category_distribution)

print("Total unique categories:", len(unique_categories_list))


Unique categories:
['ORDER' 'SHIPPING' 'CANCEL' 'INVOICE' 'PAYMENT' 'REFUND' 'FEEDBACK'
 'CONTACT' 'ACCOUNT' 'DELIVERY' 'SUBSCRIPTION']

Category distribution:
category
ACCOUNT         5986
ORDER           3988
REFUND          2992
INVOICE         1999
CONTACT         1999
PAYMENT         1998
FEEDBACK        1997
DELIVERY        1994
SHIPPING        1970
SUBSCRIPTION     999
CANCEL           950
Name: count, dtype: int64
Total unique categories: 11


In [52]:
# Convert the text in the 'instruction' and 'intent' columns to lowercase
df['instruction'] = df['instruction'].str.lower()
df['intent'] = df['intent'].str.lower()

# Display the resulting DataFrame
print(df)



                                             instruction category  \
0       question about cancelling order {{order number}}    ORDER   
1      i have a question about cancelling oorder {{or...    ORDER   
2        i need help cancelling puchase {{order number}}    ORDER   
3             i need to cancel purchase {{order number}}    ORDER   
4      i cannot afford this order, cancel purchase {{...    ORDER   
...                                                  ...      ...   
26867  i am waiting for a rebate of {{refund amount}}...   REFUND   
26868  how to see if there is anything wrong with my ...   REFUND   
26869  i'm waiting for a reimbjrsement of {{currency ...   REFUND   
26870  i don't know what to do to see my reimbursemen...   REFUND   
26871  i need to know if there is anything new on the...   REFUND   

             intent                                           response  
0      cancel_order  I've understood you have a question regarding ...  
1      cancel_order  I've

## Entity


### Placeholder


Checking placeholder


In [55]:
import pandas as pd
import re
from collections import Counter

# Detect rows with placeholders
df['has_placeholder'] = df['instruction'].str.contains(r'\{\{.*?\}\}')

# Count rows with placeholders
rows_with_placeholders = df['has_placeholder'].sum()

# Extract placeholders function
def extract_placeholders(text):
    return re.findall(r'\{\{(.*?)\}\}', text)

# Apply function and get all placeholders
placeholders = df[df['has_placeholder']]['instruction'].apply(extract_placeholders).explode()

# Count occurrences of each placeholder
placeholder_counts = Counter(placeholders)

# Convert to DataFrame
placeholder_df = pd.DataFrame.from_dict(placeholder_counts, orient='index', columns=['count'])

# Display initial results
print(f"Rows with placeholders: {rows_with_placeholders}")
print("\nInitial Distribution of Placeholders:")
print(placeholder_df.sort_values(by='count', ascending=False))

# Print total rows and missing values
print("\nTotal rows in the dataset:", len(df))
print("\nMissing values per column:")
print(df.isnull().sum())

# Remove placeholders with the lowest occurrence
if not placeholder_df.empty:
    min_count = placeholder_df['count'].min()  # Find the lowest count
    placeholder_df = placeholder_df[placeholder_df['count'] > min_count]  # Keep only placeholders with higher counts

# Display updated results
print("\nUpdated Distribution of Placeholders (After Removing Lowest Occurrences):")
print(placeholder_df.sort_values(by='count', ascending=False))


Rows with placeholders: 0

Initial Distribution of Placeholders:
Empty DataFrame
Columns: [count]
Index: []

Total rows in the dataset: 26872

Missing values per column:
instruction        0
category           0
intent             0
response           0
has_placeholder    0
ner_labels         0
dtype: int64

Updated Distribution of Placeholders (After Removing Lowest Occurrences):
Empty DataFrame
Columns: [count]
Index: []


In [ ]:
!pip install faker



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.9 MB/s eta 0:00:00


In [ ]:
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=032137c1d515880fb4279680e764e63ede60905588cd4168312e448b35a3854a
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


# Data Preprocessing


## Load Dataset


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('dataset.csv')

# Inspect the dataset
print(df.head())



                                                        re everything is in order  \
BLZ  I am waiting for a reimburswment of {{Currency ...                    REFUND   
BL   I want assistance to see the status of the comp...                    REFUND   
BLQ  i expect a rebate of {{Refund Amount}} dollars                        REFUND   
BELQ i do not know what to do to track my compensation                     REFUND   
BQ   i expect a refund of {{Currency Symbol}}{{Refun...                    REFUND   

                                                         I recommend contacting our customer support team directly. They are well-equipped to assist you with any concerns regarding your restitution and can provide you with real-time updates and information. You can reach them through our helpline at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. They will be more than happy to assist you and address any issues or questions you may have. R

## Text Cleaning


In [ ]:
# prompt: lowercase all column

# prompt: Remove special characters. include import librarie dont implement this

import re

def remove_special_characters(text):
    # Keep {{ and }}, remove other special characters
    text = re.sub(r'(?<!{){(?!{)|(?<!})}(?!})|[^\w\s{}]', '', text)
    return text

# Example usage (assuming you have a 'text' column in your DataFrame)
if 'instruction' in df.columns:
    df['instruction'] = df['instruction'].apply(remove_special_characters)
else:
    print("Column 'instruction' not found in the DataFrame.")



# Lowercase all column names
df.columns = df.columns.str.lower()

print(df.head())


Column 'instruction' not found in the DataFrame.
                                                        re everything is in order  \
BLZ  I am waiting for a reimburswment of {{Currency ...                    REFUND   
BL   I want assistance to see the status of the comp...                    REFUND   
BLQ  i expect a rebate of {{Refund Amount}} dollars                        REFUND   
BELQ i do not know what to do to track my compensation                     REFUND   
BQ   i expect a refund of {{Currency Symbol}}{{Refun...                    REFUND   

                                                         i recommend contacting our customer support team directly. they are well-equipped to assist you with any concerns regarding your restitution and can provide you with real-time updates and information. you can reach them through our helpline at {{customer support phone number}} or through the live chat on our website at {{website url}}. they will be more than happy to assist you an

## NER


In [ ]:
import pandas as pd
import re
from collections import Counter


# Detect rows with placeholders
df['has_placeholder'] = df['instruction'].str.contains(r'\{\{.*?\}\}')

# Count rows with placeholders
rows_with_placeholders = df['has_placeholder'].sum()

# Extract placeholders function
def extract_placeholders(text):
    return re.findall(r'\{\{(.*?)\}\}', text)

# Apply function and get all placeholders
placeholders = df[df['has_placeholder']]['instruction'].apply(extract_placeholders).explode()

# Count occurrences of each placeholder
placeholder_counts = Counter(placeholders)

# Convert to DataFrame
placeholder_df = pd.DataFrame.from_dict(placeholder_counts, orient='index', columns=['count'])

# Display initial results
print(f"Rows with placeholders: {rows_with_placeholders}")
print("\nInitial Distribution of Placeholders:")
print(placeholder_df.sort_values(by='count', ascending=False))

# Print total rows and missing values
print("\nTotal rows in the dataset:", len(df))
print("\nMissing values per column:")
print(df.isnull().sum())

# Remove placeholders with the lowest occurrence
if not placeholder_df.empty:
    min_count = placeholder_df['count'].min()  # Find the lowest count
    placeholder_df = placeholder_df[placeholder_df['count'] > min_count]  # Keep only placeholders with higher counts

# Display updated results
print("\nUpdated Distribution of Placeholders (After Removing Lowest Occurrences):")
print(placeholder_df.sort_values(by='count', ascending=False))


Rows with placeholders: 6670

Initial Distribution of Placeholders:
                  count
Order Number       2907
Account Type       1011
Person Name         887
Account Category    822
Refund Amount       624
Currency Symbol     372
Delivery City       234
Delivery Country    177
Invoice Number        8

Total rows in the dataset: 26872

Missing values per column:
flags              0
instruction        0
category           0
intent             0
response           0
has_placeholder    0
dtype: int64

Updated Distribution of Placeholders (After Removing Lowest Occurrences):
                  count
Order Number       2907
Account Type       1011
Person Name         887
Account Category    822
Refund Amount       624
Currency Symbol     372
Delivery City       234
Delivery Country    177


In [ ]:
import re
import pandas as pd
from faker import Faker

# Initialize Faker
fake = Faker()

# Define synthetic data generators
data_generators = {
    'order number': lambda: fake.unique.bothify(text='ORD-#####'),
    'account type': lambda: fake.random_element(elements=('Business', 'Personal')),
    'person name': lambda: fake.name(),
    'account category': lambda: fake.random_element(elements=('Category A', 'Category B', 'Category C')),
    'refund amount': lambda: f"${fake.random_number(digits=3)}.00",
    'currency symbol': lambda: fake.currency_symbol(),
    'delivery city': lambda: fake.city(),
    'delivery country': lambda: fake.country(),
    'invoice number': lambda: fake.unique.bothify(text='INV###')
}

# Function to replace placeholders and track NER labels
def replace_placeholders(text, data_generators):
    if pd.isnull(text):
        return text, []  # Return empty list for NER labels

    placeholder_pattern = re.compile(r'\{\{(.*?)\}\}')  # Match placeholders {{...}}
    placeholders = placeholder_pattern.findall(text)
    ner_labels = []  # Store entity labels

    for placeholder in placeholders:
        placeholder_cleaned = placeholder.strip().lower()  # Normalize case
        if placeholder_cleaned in data_generators:
            replacement_value = data_generators[placeholder_cleaned]()  # Generate synthetic data
            text = text.replace(f'{{{{{placeholder}}}}}', replacement_value, 1)  # Replace placeholder

            # Append entity info in required format
            ner_labels.append({'text': replacement_value, 'label': placeholder_cleaned.replace(" ", "_")})

    return text, ner_labels

# Apply the placeholder replacement function
df[['instruction', 'ner_labels']] = df['instruction'].apply(
    lambda x: pd.Series(replace_placeholders(x, data_generators))
)

# Ensure all rows are retained, even if ner_labels is empty
df['ner_labels'] = df.apply(lambda row: row['ner_labels'] if row['has_placeholder'] else [], axis=1)

print(df.head())


   flags                                        instruction category  \
0      B          question about cancelling order ORD-74965    ORDER   
1    BQZ  i have a question about cancelling oorder ORD-...    ORDER   
2   BLQZ           i need help cancelling puchase ORD-94215    ORDER   
3     BL                I need to cancel purchase ORD-84946    ORDER   
4  BCELN  I cannot afford this order cancel purchase ORD...    ORDER   

         intent                                           response  \
0  cancel_order  I've understood you have a question regarding ...   
1  cancel_order  I've been informed that you have a question ab...   
2  cancel_order  I can sense that you're seeking assistance wit...   
3  cancel_order  I understood that you need assistance with can...   
4  cancel_order  I'm sensitive to the fact that you're facing f...   

   has_placeholder                                        ner_labels  
0             True  [{'text': 'ORD-74965', 'label': 'order_number'}]  
1   

## Function to Create BIO Tags


In [ ]:
import re

# Function to generate BIO tags
def generate_bio_tags(instruction, ner_labels):
    # Tokenize the instruction into words using regex to handle punctuation
    tokens = re.findall(r'\w+|[^\w\s]', instruction)

    # Initialize BIO tags as 'O' (Outside) for all tokens
    bio_tags = ['O'] * len(tokens)

    # Process each entity in ner_labels
    for entity in ner_labels:
        entity_text = entity['text']
        entity_label = entity['label']

        # Tokenize the entity text using the same regex
        entity_tokens = re.findall(r'\w+|[^\w\s]', entity_text)

        # Find the start index of the entity in the instruction
        try:
            # Create a sliding window to find the entity tokens in the instruction tokens
            for i in range(len(tokens) - len(entity_tokens) + 1):
                if tokens[i:i + len(entity_tokens)] == entity_tokens:
                    # Assign BIO tags
                    for j in range(len(entity_tokens)):
                        if j == 0:
                            bio_tags[i + j] = f"B-{entity_label}"  # Beginning of the entity
                        else:
                            bio_tags[i + j] = f"I-{entity_label}"  # Inside the entity
                    break  # Stop after finding the first match
        except Exception as e:
            # If the entity text is not found in the tokens, log the error
            print(f"Error processing entity '{entity_text}' in instruction: {instruction}")
            continue

    return tokens, bio_tags

# Apply the BIO tag generation function
df['bio_tags'] = df.apply(lambda row: generate_bio_tags(row['instruction'], row['ner_labels']), axis=1)

# Split the result into separate columns for tokens and tags
df[['tokens', 'tags']] = pd.DataFrame(df['bio_tags'].tolist(), index=df.index)

# Drop the intermediate 'bio_tags' column
df.drop(columns=['bio_tags'], inplace=True)

# Display the final DataFrame
print(df[['instruction', 'ner_labels', 'tokens', 'tags']])


                                             instruction  \
0              question about cancelling order ORD-74965   
1      i have a question about cancelling oorder ORD-...   
2               i need help cancelling puchase ORD-94215   
3                    I need to cancel purchase ORD-84946   
4      I cannot afford this order cancel purchase ORD...   
...                                                  ...   
26867       I am waiting for a rebate of $428.00 dollars   
26868  how to see if there is anything wrong with my ...   
26869        Im waiting for a reimbjrsement of VT$311.00   
26870  I dont know what to do to see my reimbursement...   
26871  I need to know if there is anything new on the...   

                                              ner_labels  \
0       [{'text': 'ORD-74965', 'label': 'order_number'}]   
1       [{'text': 'ORD-78422', 'label': 'order_number'}]   
2       [{'text': 'ORD-94215', 'label': 'order_number'}]   
3       [{'text': 'ORD-84946', 'label':

In [ ]:

df.to_csv('output.csv', index=False)


## Label Encoding


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load your dataset
df = pd.read_csv('output.csv')

# Display the first few rows of the dataset
print("Original Dataset:")
print(df.head())

# Initialize LabelEncoders
intent_encoder = LabelEncoder()
category_encoder = LabelEncoder()

# Fit and transform the labels
df['intent_encoded'] = intent_encoder.fit_transform(df['intent'])
df['category_encoded'] = category_encoder.fit_transform(df['category'])

# Save the encoded dataset (optional)
df.to_csv('customer_support_data_encoded.csv', index=False)


Original Dataset:
   flags                                        instruction category  \
0      B          question about cancelling order ORD-74965    ORDER   
1    BQZ  i have a question about cancelling oorder ORD-...    ORDER   
2   BLQZ           i need help cancelling puchase ORD-94215    ORDER   
3     BL                I need to cancel purchase ORD-84946    ORDER   
4  BCELN  I cannot afford this order cancel purchase ORD...    ORDER   

         intent                                           response  \
0  cancel_order  I've understood you have a question regarding ...   
1  cancel_order  I've been informed that you have a question ab...   
2  cancel_order  I can sense that you're seeking assistance wit...   
3  cancel_order  I understood that you need assistance with can...   
4  cancel_order  I'm sensitive to the fact that you're facing f...   

   has_placeholder                                        ner_labels  \
0             True  [{'text': 'ORD-74965', 'label': 'ord

## Splitting of Dataset


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('customer_support_data_encoded.csv')

# Prepare for splitting
df['stratify_key'] = df[['category', 'intent']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

# Split the dataset
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df['stratify_key'])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data['stratify_key'])

# Drop stratify column (no longer needed)
train_data.drop(columns=['stratify_key'], inplace=True)
val_data.drop(columns=['stratify_key'], inplace=True)
test_data.drop(columns=['stratify_key'], inplace=True)

# Export datasets (not tokenized or annotated yet)
train_data.to_csv('train_data.csv', index=False)
val_data.to_csv('val_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

# Print the first few rows of the train dataset to verify
print(train_data.head())


      flags                                        instruction  category  \
7119    BIL    how to lodge a reclamation against your company  FEEDBACK   
14235   BKL                       modifying Category A account   ACCOUNT   
14418   BKL                    update data on standard account   ACCOUNT   
13162    BL  I need assistance seeing how soon can I expect...  DELIVERY   
1626    BLM    I need to swap some items of purchase ORD-75449     ORDER   

                intent                                           response  \
7119         complaint  I'm truly sorry to hear that you have encounte...   
14235     edit_account  How splendid it is to embark on the extraordin...   
14418     edit_account  How invigorating it is to witness your proacti...   
13162  delivery_period  We understand your eagerness to track and know...   
1626      change_order  We understand that you need to swap some items...   

       has_placeholder                                         ner_labels  \
711

Sample only


In [ ]:
import pandas as pd

# Load the dataset
train_data = pd.read_csv('train_data.csv')

# Ensure the dataset has at least 300 rows
if len(train_data) >= 300:
    # Split into three parts
    train_data.iloc[:100].to_csv('train_data_part1.csv', index=False)
    train_data.iloc[100:200].to_csv('train_data_part2.csv', index=False)
    train_data.iloc[200:300].to_csv('train_data_part3.csv', index=False)
    print("Dataset successfully split into three parts.")
else:
    print("Dataset does not have at least 300 rows.")


Dataset successfully split into three parts.
